In [30]:
from typing import Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.graph import END

from dotenv import load_dotenv
import os

load_dotenv()

# 메모리 저장소 설정
memory = MemorySaver()


# 메시지 상태와 요약 정보를 포함하는 상태 클래스
class State(MessagesState):
    messages: Annotated[list, add_messages]
    summary: str


# 대화 및 요약을 위한 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 도구 생성

In [45]:
from langchain.agents import Tool
from langchain.tools import StructuredTool
from notion.planner import Planner
from notion.task import Task
from pydantic import BaseModel, Field

planner = Planner()

In [46]:
class ShowTasksSchema(BaseModel):
    pass

def show_tasks():
    return planner.show_tasks()

class CreateTaskSchema(BaseModel):
    name: str = Field("No name", description="Task name")
    date: str = Field("2000-01-01 00:00", description="The date of the task, either format:\n- Single datetime: 'YYYY-MM-DD HH:MM'\n- Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'")
    group: str = Field("no group", description="Task group, e.g., '일상', '이벤트'")

def create_task(name, date, group):
    try:
        date_count = date.count(":")
        if date_count == 1:
            date = {"start": date, "end": None}
        elif date_count == 2:
            part = date.split(" ")
            start = f"{part[0]} {part[1]}"
            end = f"{part[2]} {part[3]}"
            date = {"start": start, "end": end}
        else:
            raise ValueError("Invalid date format. Use 'YYYY-MM-DD HH:MM' or 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'.")
    except ValueError as e:
        raise ValueError(f"Error parsing date: {e}")

    task = Task(task_id=None, name=name, date=date, group=group)
    return planner.add_task(task)

class DeleteTaskSchema(BaseModel):
    task_id: str = Field(..., description="The ID of the task to delete")

def delete_task(task_id):
    return planner.delete_task(task_id)

class EditTaskSchema(BaseModel):
    task_id: str = Field(..., description="The ID of the task to edit")
    name: str = Field("No name", description="New task name")
    date: str = Field("2000-01-01 00:00", description="New date of the task, either format:\n- Single datetime: 'YYYY-MM-DD HH:MM'\n- Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'")
    group: str = Field("no group", description="New task group, e.g., '일상', '이벤트'")

def edit_task(task_id, name, date, group):
    new_task = Task(task_id=task_id, name=name, date=date, group=group)
    return planner.edit_task(task_id, new_task)

In [47]:
tools = [
    StructuredTool.from_function(
        func=show_tasks,
        name="ShowTasks",
        description="Show all the tasks.",
        args_schema=ShowTasksSchema
    ),
    StructuredTool.from_function(
        func=create_task,
        name="CreateTask",
        description="Create/Add a new task in Notion.",
        args_schema=CreateTaskSchema
    ),
    StructuredTool.from_function(
        func=delete_task,
        name="DeleteTask",
        description="Delete a task from Notion. Input: Notion page ID.",
        args_schema=DeleteTaskSchema
    ),
    StructuredTool.from_function(
        func=edit_task,
        name="EditTask",
        description="Update a task's title in Notion. Input: page ID and new title.",
        args_schema=EditTaskSchema
    )
]

In [48]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    model=model,
    tools=tools,
    # checkpointer=memory
)

In [49]:
def print_stream(stream):
    for chunk in stream:
        message = chunk["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [50]:
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [
    ("user", "how much tasks did you saw?")
]}
print_stream(agent_executor.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

how much tasks did you saw?
================================== Ai Message ==================================
Tool Calls:
  ShowTasks (call_sWSrzjluZtliGGch6P0E4MGT)
 Call ID: call_sWSrzjluZtliGGch6P0E4MGT
  Args:
================================= Tool Message =================================
Name: ShowTasks

[1d084e0b-d49f-80cc-a47a-f7bf87fba35e][연신내 카페][2025-04-10 12:30 ~ 2025-04-10 15:00][일과]
[1d084e0b-d49f-806b-8497-c287550c1450][연신내 점심][2025-04-10 12:00 ~ 2025-04-10 12:30][일과]
[1ce84e0b-d49f-80b2-997f-f530536cef61][점심먹고 불광천 산책][2025-04-09 15:00][]
[1ce84e0b-d49f-8067-89f1-ddf765db811c][던전헬퍼 Github README 채우기][2025-04-09 12:59][이벤트]
[1ce84e0b-d49f-80ea-ac12-c29707014aa1][스마트가로등 Github README 채우기][2025-04-08 12:59][이벤트]
[1cc84e0b-d49f-8173-a2e0-c9c83841a781][edit_test][2025-04-05 07:07][test]
[1c284e0b-d49f-8168-8e9c-c70d4c8496d2][Squarekur 회의][2025-03-30 22:00][일과]
[df4caa92-6fbd-4ffa-9d08-3887b453558

In [ ]:
config = {"configurable": {"thread_id": "1"}}
inputs = [
    "create a task named 'Squarekur 회의', 2025-06-15 22:00 in group '일상'"
]
for input_text in inputs:
    inputs = {"messages": [
        ("user", input_text)
    ]}
    print_stream(agent_executor.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

create a task named 'Squarekur 회의', 2025-06-15 22:00 in group '일상'
================================== Ai Message ==================================
Tool Calls:
  CreateTask (call_WPwil70dqcgeesqYgAUOnsHR)
 Call ID: call_WPwil70dqcgeesqYgAUOnsHR
  Args:
    name: Squarekur 회의
    date: 2025-06-15 22:00
    group: 일상
================================= Tool Message =================================
Name: CreateTask

Task[21284e0b-d49f-812d-b6ec-cf2032a8d9e8] added successfully!
================================== Ai Message ==================================

The task "Squarekur 회의" has been successfully created for June 15, 2025, at 22:00 in the group "일상."
